In [18]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from torch import Tensor
from torch.hub import load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional

In [ ]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

define value tracker

In [ ]:
def value_tracker(value_plot, value, num):
  '''num, loss_value, are Tensor'''
  vis.line(X=num,
           Y=value,
           vin = value_plot,
           update = 'append')

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))

How to Calculate mean and std in Normalize

In [12]:
transform = transforms.Compose([
  transforms.ToTensor()                                
])

trainset = torchvision.datasets.CIFAR10(root = './data', train=True, download = True, transform = transform)

print(trainset.data.shape)

train_data_mean = trainset.data.mean(axis=(0,1,2))
train_data_std = trainset.data.std(axis=(0,1,2))

print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean /255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [14]:
transform_train = transforms.Compose([
  transforms.RandomCrop(32, padding = 4),
  transforms.ToTensor(),
  transforms.Normalize(train_data_mean, train_data_std)                                      
])

transform_test = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(train_data_mean, train_data_std)                                         
])

trainset = torchvision.datasets.CIFAR10(root = './data', train=True, download = True, transform = transform_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 256, shuffle = True, num_workers = 4)

testset = torchvision.datasets.CIFAR10(root = './data', train=False, download = True, transform = transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size = 256, shuffle = False, num_workers = 4)

classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog','horse','ship','truck')


Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


make ResNet50

In [15]:
import torchvision.models.resnet as resnet

In [16]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [19]:
class ResNet(nn.Module):
   
    # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) #ResNet50

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:

        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 16
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        #inputs = 3x224x224
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
                               bias=False)
        #outputs = self.conv1(inputs)
        #outputs.shape = [64x112x112]

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        #inputs = [64x112x112]
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        #outputs.shape = [64x56x56]

        self.layer1 = self._make_layer(block, 16, layers[0]) #layers[0]=3
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1,
                                       dilate=replace_stride_with_dilation[0]) #layers[1]=4
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1]) #layers[2]=6
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2]) #layers[3]=3
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]
    
    #self.inplanes = 64
    #self.layer1 = self._make_layer(Bottleneck, 64, 3) #layers[0]=3
    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1

        if stride != 1 or self.inplanes != planes * block.expansion: #64 != 64 * 4(Bottleneck.expansion)
            
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), #conv1x1(64, 64*4, 1)
                norm_layer(planes * block.expansion), #norm(256)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        #layers.append(Bottleneck(64,64,1, downsample, self.groups, self.base_width, previous_dilation, norm_layer))

        self.inplanes = planes * block.expansion #self.inplanes = 64*4 = 256
        

        for _ in range(1, blocks):
        #for _ in range(1, 3): #2번
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    #self.layer1 = [
      #Bottleneck(64,64,1, downsample),
      #Bottlenek(256, 64),
      #Bottleneck(256, 64)
    #]
    #self.layer2 = [
      #Bottleneck(256,128,2, downsample),
      #Bottlenek(512, 128),
      #Bottleneck(512, 128),
      #Bottleneck(512, 128)
    #]

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        #x.shape=[batch, 3x32x32]
        x = self.conv1(x)
        #x.shape=[1, 16,32,32]
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)

        x = self.layer1(x)
        #x.shape=[1,128,32,32]
        x = self.layer2(x)
        #x.shape=[1,256,32,32]
        x = self.layer3(x)
        #x.shape=[1,512,16,16]
        x = self.layer4(x)
        #x.shape=[1,1024,8,8]

        x = self.avgpool(x) #global avage pooling으로 fc대신
        x = torch.flatten(x, 1) #<=> x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [20]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3],10, True)

In [21]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

모델 테스트

In [22]:
a = torch.Tensor(1,3,32,32)#.to(device)
out = resnet50(a)
print(out)

tensor([[ 0.0338,  0.0122,  0.0168, -0.0428, -0.0382, -0.0435, -0.0441,  0.0176,
         -0.0341, -0.0110]], grad_fn=<AddmmBackward0>)


In [23]:
criterion = nn.CrossEntropyLoss()#.to(device)
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 00.1, momentum=0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.5)

make plot

In [ ]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(tiltle='Accuracy', legend=['Acc'], showlegend=True))

define acc_check function

In [24]:
def acc_check(net, test_set, epoch, save = 1):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_set:
      images, labels = data
      images = images #.to(device)
      labels = labels #.to(device)
      
      outputs = net(images)

      _, predicted = torch.max(outputs.data, axis = 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  acc = (100* correct/total)
  print('Accuracy of the network on the 10000 test images: %d %%'% acc)
  if save:
    torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
  return acc

Training with (acc check + model save)

In [ ]:
print(len(trainloader))
epochs = 150

for epoch in range(epochs):
  running_loss = 0.0
  lr_sche.step()
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs = inputs#.to(device)
    labels = labels#.to(device)

    #zero the parameter gradients
    optimizer.zero_grad()

    #forward + backward + optimize
    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    #print statistics
    running_loss += loss.item()
    if i % 30 == 29: #30번마다 print, loss_tracker
      #loss_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i+epoch*len(trainloader)]))
      print('[%d, %5d] loss: %.3f' %(epoch+1, i+1, running_loss/30))
      running_loss = 0.0
    
  #epoch마다 check Accuracy 
  acc = acc_check(resnet50, testloader, epoch, save = 1)
  #value_tracker(acc_plt, torch.Tensor([acc]), torch.Tesor([epoch]))
print('Finished Training')